In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], 
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    except RuntimeError as e:
        print(e)

In [11]:
data_dir = 'Food_image/images/'

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## ***Data Collection***

In [13]:
datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)

In [14]:
train_data = datagen.flow_from_directory(data_dir, target_size = (250,250), batch_size = 64, class_mode = 'categorical',
                                        subset = 'training')

val_data = datagen.flow_from_directory(data_dir, target_size = (250,250), batch_size = 64, class_mode = 'categorical',
                                        subset = 'validation')

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [15]:
import tensorflow_hub as hub
ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'
Efficientnet_b0 = "https://tfhub.dev/google/efficientnet/b0/classification/1"
Efficientnet_b7 = "https://tfhub.dev/google/efficientnet/b7/classification/1"

In [7]:
%load_ext tensorboard
%tensorboard --logdir logs --port 6006

Reusing TensorBoard on port 6006 (pid 70256), started 13:14:40 ago. (Use '!kill 70256' to kill it.)

## ***ResNet Model Building***

In [8]:
model_ResNet = tf.keras.Sequential([
    hub.KerasLayer(ResNet_V2_50, trainable = False, input_shape = (250,250,3), name = 'Resnet_V2_50'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax', name = 'Output_layer')
])

model_ResNet.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [9]:
model_ResNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Resnet_V2_50 (KerasLayer)   (None, 1001)              25615849  
                                                                 
 flatten (Flatten)           (None, 1001)              0         
                                                                 
 Output_layer (Dense)        (None, 101)               101202    
                                                                 
Total params: 25,717,051
Trainable params: 101,202
Non-trainable params: 25,615,849
_________________________________________________________________


In [10]:
resnet_model = model_ResNet.fit(train_data, epochs = 10, verbose = 1) #5

Epoch 1/10
632/632 [==============================] - 1158s 2s/step - loss: 2.2293 - accuracy: 0.4602
Epoch 2/10
632/632 [==============================] - 263s 416ms/step - loss: 1.7440 - accuracy: 0.5604
Epoch 3/10
632/632 [==============================] - 263s 415ms/step - loss: 1.6266 - accuracy: 0.5849
Epoch 4/10
632/632 [==============================] - 263s 416ms/step - loss: 1.5588 - accuracy: 0.5976
Epoch 5/10
632/632 [==============================] - 263s 416ms/step - loss: 1.5073 - accuracy: 0.6112
Epoch 6/10
632/632 [==============================] - 263s 416ms/step - loss: 1.4737 - accuracy: 0.6178
Epoch 7/10
632/632 [==============================] - 263s 416ms/step - loss: 1.4375 - accuracy: 0.6244
Epoch 8/10
632/632 [==============================] - 264s 418ms/step - loss: 1.4108 - accuracy: 0.6306
Epoch 9/10
632/632 [==============================] - 264s 418ms/step - loss: 1.3887 - accuracy: 0.6373
Epoch 10/10
632/632 [==============================] - 266s 420ms/

In [11]:
model_ResNet.evaluate(val_data)

158/158 [==============================] - 422s 3s/step - loss: 1.7935 - accuracy: 0.5684


[1.7935258150100708, 0.5684158205986023]

## ***Efficientnet_B0 Model Building***

In [8]:
model_Efficientnet = tf.keras.Sequential([
    hub.KerasLayer(Efficientnet_b0, trainable = False, input_shape = (250,250,3), name = 'Resnet_V2_50'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax', name = 'Output_layer')
])

model_Efficientnet.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

model_Efficientnet.summary()


#define callback
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import datetime,os

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


#1.ModelCheckpoint
filepath = os.path.join("logs", "weight.hdf5")
mc = ModelCheckpoint(
    filepath, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='auto'
)

#2.EarlyStopping
#es=EarlyStopping(monitor='loss', patience=10, verbose=1)

#3.ReduceLROnPlateau
rp=ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='auto',
    min_delta=0.001,
    cooldown=0,
    min_lr=0
)


efficientnet_model = model_Efficientnet.fit(train_data, epochs = 100, verbose = 1,validation_data=val_data,callbacks=[mc,rp,tb])

model_Efficientnet.save("Efficientnet_b0_{}.h5".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
model_Efficientnet.save_weights('Efficientnet_b0_weight_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

model_Efficientnet.evaluate(val_data)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Resnet_V2_50 (KerasLayer)   (None, 1000)              5330564   
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 Output_layer (Dense)        (None, 101)               101101    
                                                                 
Total params: 5,431,665
Trainable params: 101,101
Non-trainable params: 5,330,564
_________________________________________________________________


## ***Comparison Between Both Models***

In [ ]:
def plot_graph(history, history_1):
    loss_res = history.history['loss']
    loss_ef = history_1.history['loss']
    
    Accuracy_res = history.history['accuracy']
    Accuracy_ef = history_1.history['accuracy']

    epochs = range(len(history.history['loss']))

    plt.plot(epochs, loss_res, label = 'ResNet Loss')
    plt.plot(epochs, loss_ef, label = 'Efficientnet Loss')
    plt.title('Epochs Vs Loss')
    plt.xlabel('epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.figure()
    plt.plot(epochs, Accuracy_res, label = 'ResNet Accuracy')
    plt.plot(epochs, Accuracy_ef, label = 'Efficientnet Accuracy')
    plt.title('Epochs Vs Accuracy')
    plt.xlabel('epochs')
    plt.ylabel('Accuracy')
    plt.legend()

plot_graph(resnet_model, efficientnet_model)

## ***Efficientnet_B7 Model Building***

In [16]:
model_Efficientnet_b7 = tf.keras.Sequential([
    hub.KerasLayer(Efficientnet_b7, trainable = False, input_shape = (250,250,3), name = 'Resnet_V2_50'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax', name = 'Output_layer')
])

model_Efficientnet_b7.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

model_Efficientnet_b7.summary()


#define callback
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import datetime,os

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


#1.ModelCheckpoint
filepath = os.path.join("logs", "weight.hdf5")
mc = ModelCheckpoint(
    filepath, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='auto'
)

#2.EarlyStopping
es=EarlyStopping(monitor='val_loss', patience=20, verbose=1)

#3.ReduceLROnPlateau
rp=ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='auto',
    min_delta=0.001,
    cooldown=0,
    min_lr=0
)


efficientnet_b7_model = model_Efficientnet_b7.fit(train_data, epochs = 100, verbose = 1,validation_data=val_data,callbacks=[mc,rp,tb,es])

model_Efficientnet_b7.save("Efficientnet_b7_{}.h5".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
model_Efficientnet_b7.save_weights('Efficientnet_b7_weight_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

model_Efficientnet_b7.evaluate(val_data)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Resnet_V2_50 (KerasLayer)   (None, 1000)              66658680  
                                                                 
 flatten_2 (Flatten)         (None, 1000)              0         
                                                                 
 Output_layer (Dense)        (None, 101)               101101    
                                                                 
Total params: 66,759,781
Trainable params: 101,101
Non-trainable params: 66,658,680
_________________________________________________________________
Epoch 1/100
1263/1263 [==============================] - ETA: 0s - loss: 1.7384 - accuracy: 0.5631
Epoch 00001: val_loss improved from inf to 1.43895, saving model to logs\weight.hdf5
1263/1263 [==============================] - 806s 621ms/step - loss: 1.7384 - accuracy: 0.5631 - val_loss: 1.4389 - 

KeyboardInterrupt: 